In [1]:
import os
import numpy as np
import cv2
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats

from tqdm.notebook import tqdm
from tqdm import tqdm

In [6]:
fps = 120
dataframes = {}

folder_path = 'videos/all_videos'
file_names = os.listdir(folder_path)

for video_name in file_names:
    cap = cv2.VideoCapture('videos/all_videos/' + video_name) # vervang dit path naar het path van je eigen video
    cap_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # The following code is modified code written by Sten den Hartog!
    ###################################################################

    # Set up parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners=1, qualityLevel=0.3, minDistance=7, blockSize=7)

    # Set up parameters for Lucas-Kanade optical flow
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Read the first frame
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # Select ROI for tracking
    bbox = cv2.selectROI(old_frame, False)
    cv2.destroyAllWindows()
    x, y, w, h = bbox

    # if you want to manually set the ROI, comment the lines above and paste the ROI below
    # x, y, w, h = 0, 0, 0, 0

    roi_mask = np.zeros_like(old_gray)
    roi_mask[y:y+h, x:x+w] = 255

    # Detect initial points in ROI
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=roi_mask, **feature_params)

    # Check if any points were detected
    if p0 is None:
        print("Unable to determine tracking features, please adjust bounding box.")
        cap.release()  # Release the video capture object
        sys.exit()  # Exit the program

    # List to store the displacement of points
    displacements = []

    output = cv2.VideoWriter( 
            'C:/Users/Gebruiker/Videos/roi_tracked_output.avi' , cv2.VideoWriter_fourcc(*'XVID'), fps, (cap_width, cap_height))

    # Draw initial tracking point
    highlight_color = (0, 255, 0)  # Green
    circle_radius = 5              
    circle_thickness = -1          # -1 to fill the circle

    frame_circle = cv2.circle(old_frame, p0.astype(int).ravel(), circle_radius, highlight_color, circle_thickness)
    output.write(frame_circle)

    # Process video
    iteration = 0
    with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc="Tracking") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Calculate optical flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

            # Ensure optical flow was successful
            if p1 is not None and len(p1) > 0:
                # Select good points
                good_new = p1[st == 1]
                good_old = p0[st == 1]

                # Store points' displacements
                for i, (new, old) in enumerate(zip(good_new, good_old)):
                    a, b = new.ravel()
                    c, d = old.ravel()
                    displacements.append({'iteration': iteration, 'time': pbar.n / fps, 'id': i, 'x': a, 'y': b, 'dx': a - c, 'dy': b - d})

                # Update the previous frame and points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
            else:
                print("Unable to determine tracking features, please adjust bounding box.")
                break  # Break out of the loop if tracking fails
            
            frame_circle = cv2.circle(frame, good_new.astype(int).ravel(), circle_radius, highlight_color, circle_thickness)
            output.write(frame_circle)

            pbar.update(1)
            iteration += 1

    cap.release()
    output.release()
    ###################################################################

    # Convert the list of displacements to a DataFrame
    df_displacements = pd.DataFrame(displacements)
    dataframes[video_name] = df_displacements

Tracking: 925it [00:03, 251.43it/s]
Tracking: 934it [00:03, 261.33it/s]
Tracking: 917it [00:03, 232.67it/s]
Tracking: 917it [00:03, 241.24it/s]
Tracking: 917it [00:03, 243.69it/s]
Tracking: 920it [00:05, 174.40it/s]
Tracking: 939it [00:06, 138.25it/s]
Tracking: 909it [00:06, 134.68it/s]
Tracking: 917it [00:04, 208.71it/s]
Tracking: 937it [00:04, 206.22it/s]
Tracking: 917it [00:04, 225.65it/s]
Tracking: 933it [00:04, 232.56it/s]
Tracking: 916it [00:04, 214.21it/s]
Tracking: 914it [00:04, 221.23it/s]
Tracking: 927it [00:04, 190.87it/s]
Tracking: 929it [00:04, 196.43it/s]
Tracking: 926it [00:04, 210.18it/s]
Tracking: 934it [00:04, 202.86it/s]
Tracking: 942it [00:04, 204.05it/s]
Tracking: 940it [00:08, 113.79it/s]


In [7]:
# create a folder for the CSVs if it doesn't already exist
output_folder = 'exported_csvs'
os.makedirs(output_folder, exist_ok=True)

# save each DataFrame to a CSV file
for video_name, df in dataframes.items():
    csv_filename = os.path.join(output_folder, f"{video_name}.csv")
    df.to_csv(csv_filename, index=False) 
    print(f"Saved: {csv_filename}")

Saved: exported_csvs\koel_640_480_120fps_take1.h264.csv
Saved: exported_csvs\koel_640_480_120fps_take2.h264.csv
Saved: exported_csvs\koel_640_480_120fps_take3.h264.csv
Saved: exported_csvs\koel_640_480_120fps_take4.h264.csv
Saved: exported_csvs\koel_640_480_120fps_take5.h264.csv
Saved: exported_csvs\koel_take1.h264.csv
Saved: exported_csvs\koel_take2.h264.csv
Saved: exported_csvs\koel_take3.h264.csv
Saved: exported_csvs\koel_take4.h264.csv
Saved: exported_csvs\koel_take5.h264.csv
Saved: exported_csvs\vries_640_480_120fps_take1.h264.csv
Saved: exported_csvs\vries_640_480_120fps_take2.h264.csv
Saved: exported_csvs\vries_640_480_120fps_take3.h264.csv
Saved: exported_csvs\vries_640_480_120fps_take4.h264.csv
Saved: exported_csvs\vries_640_480_120fps_take5.h264.csv
Saved: exported_csvs\vries_take1.h264.csv
Saved: exported_csvs\vries_take2.h264.csv
Saved: exported_csvs\vries_take3.h264.csv
Saved: exported_csvs\vries_take4.h264.csv
Saved: exported_csvs\vries_take5.h264.csv


In [13]:
# load dataframes from csv files
dataframes = {}
output_folder = 'exported_csvs'

# list all CSV files in the output folder
csv_files = [f for f in os.listdir(output_folder) if f.endswith('.csv')]

# load each CSV file into the dictionary
for csv_file in csv_files:
    video_name = os.path.splitext(csv_file)[0]  # Get the video name without the extension
    csv_path = os.path.join(output_folder, csv_file)
    dataframes[video_name] = pd.read_csv(csv_path)
    print(f"Loaded: {csv_path}")


Loaded: exported_csvs\koel_640_480_120fps_take1.h264.csv
Loaded: exported_csvs\koel_640_480_120fps_take2.h264.csv
Loaded: exported_csvs\koel_640_480_120fps_take3.h264.csv
Loaded: exported_csvs\koel_640_480_120fps_take4.h264.csv
Loaded: exported_csvs\koel_640_480_120fps_take5.h264.csv
Loaded: exported_csvs\koel_take1.h264.csv
Loaded: exported_csvs\koel_take2.h264.csv
Loaded: exported_csvs\koel_take3.h264.csv
Loaded: exported_csvs\koel_take4.h264.csv
Loaded: exported_csvs\koel_take5.h264.csv
Loaded: exported_csvs\vries_640_480_120fps_take1.h264.csv
Loaded: exported_csvs\vries_640_480_120fps_take2.h264.csv
Loaded: exported_csvs\vries_640_480_120fps_take3.h264.csv
Loaded: exported_csvs\vries_640_480_120fps_take4.h264.csv
Loaded: exported_csvs\vries_640_480_120fps_take5.h264.csv
Loaded: exported_csvs\vries_take1.h264.csv
Loaded: exported_csvs\vries_take2.h264.csv
Loaded: exported_csvs\vries_take3.h264.csv
Loaded: exported_csvs\vries_take4.h264.csv
Loaded: exported_csvs\vries_take5.h264.csv


Dominant frequency appears to change depending on how many rows we take from the df? TODO

In [14]:
df_1sec = dataframes['koel_640_480_120fps_take1.h264'].head(100)
# Compute the FFT of the y-displacement
time = df_1sec['time'].values
y = df_1sec['dy'].values

# Time step
dt = np.mean(np.diff(time))
# Sampling frequency
fs = 1 / dt  

fft_result = np.fft.fft(y)
frequencies = np.fft.fftfreq(len(y), d=dt)

# Only keep positive frequencies
positive_freqs = frequencies[frequencies >= 0]
amplitude_spectrum = np.abs(fft_result[frequencies >= 0])
dominant_frequency = positive_freqs[np.argmax(amplitude_spectrum)]
dominant_frequency

43.199999999999996

In [15]:
# plot the frequency spectrum for each video in the same plot
fig = go.Figure()

for video_name, df_displacements in dataframes.items():
    # Compute the FFT of the y-displacement
    time = df_displacements['time'].values
    y = df_displacements['dy'].values

    # Time step
    dt = np.mean(np.diff(time))
    # Sampling frequency
    fs = 1 / dt  

    fft_result = np.fft.fft(y)
    frequencies = np.fft.fftfreq(len(y), d=dt)

    # Only keep positive frequencies
    positive_freqs = frequencies[frequencies >= 0]
    amplitude_spectrum = np.abs(fft_result[frequencies >= 0])

    color = 'red' if 'koel' in video_name else 'blue'
    fig.add_trace(
        go.Scatter(
            x=positive_freqs,
            y=amplitude_spectrum,
            mode='lines',
            name=video_name,
            line=dict(color=color)
        )
    )

fig.update_layout(
    title='Frequency Spectrum for All Videos',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Amplitude',
    legend_title='Videos'
)

fig.show()

In [16]:
# plot the cooling mode (koel or vries) vs the dominant frequency of the frequency spectrum
fig = go.Figure()

cooling_modes = []
dominant_frequencies = []
for video_name, df_displacements in dataframes.items():
    # Compute the FFT of the y-displacement
    time = df_displacements['time'].values
    y = df_displacements['dy'].values

    # Time step
    dt = np.mean(np.diff(time))
    # Sampling frequency
    fs = 1 / dt  

    fft_result = np.fft.fft(y)
    frequencies = np.fft.fftfreq(len(y), d=dt)

    # Only keep positive frequencies
    positive_freqs = frequencies[frequencies >= 0]
    amplitude_spectrum = np.abs(fft_result[frequencies >= 0])

    # find cooling mode from video_name
    cooling_mode = 'koel' if 'koel' in video_name else 'vries'
    cooling_modes.append(cooling_mode)
    # find the dominant frequency
    dominant_frequency = positive_freqs[np.argmax(amplitude_spectrum)]
    dominant_frequencies.append(dominant_frequency)

# plot
plot_df = pd.DataFrame({
'Cooling Mode': cooling_modes,
'Dominant Frequency (Hz)': dominant_frequencies
})

fig = px.scatter(
    plot_df,
    x='Dominant Frequency (Hz)',
    y='Cooling Mode',
    color='Cooling Mode',
    title='Cooling Mode vs. Dominant Frequency',
    labels={'Cooling Mode': 'Cooling Mode', 'Dominant Frequency (Hz)': 'Dominant Frequency (Hz)'},
    color_discrete_map={'koel': 'red', 'vries': 'blue'}
)

fig.show()
